In [ ]:
import math

from data_util.db_util import *
import pandas as pd
import datetime

now = datetime.datetime.now()
date = now.strftime('%m%d')
dateindex = now.strftime('%Y-%m-%d')
print(dateindex, date)
file = '../temp/Table{}.xls'.format(date)


In [ ]:
from data_util.db_util import *
import pandas as pd
import datetime
file = '../temp/Table0107.xls'

df = pd.read_csv(file, encoding='gbk', sep='\t')
df['涨幅'] = df['涨幅'].str.strip('%')
df['换手'] = df['换手'].str.strip('%')
print(df.columns)



In [ ]:
# 更新db 数据

def ts_code(code):
    if len(code) == 8:
        return f'{code[2:]}.{code[:2]}'
    else:
        return ''

conn = sqlite3.connect('../stocks.db')


for i,row in df.iterrows():
    code = row.loc['代码']
    bk = row.loc['细分行业']
    pe = row.loc['TTM市盈率']
    pb = row.loc['市净率']
    # print(code, bk, pe, pb)
    if math.isnan(pe):
        pe = 0
    if math.isnan(pb):
        pb = 0
    tc = ts_code(code)
    if len(tc)==9:
        conn.execute('''update STOCKBASIC set pe={}, pb={}, bk='{}' where ts_code = '{}' '''.format(pe,pb,bk, tc))
        # conn.commit()
        # print(f'update {i} {tc} {pe} {pb} {bk}')
    # break
conn.commit()
conn.close()
print('update finish')



In [ ]:
# 查询板块

conn = sqlite3.connect('../stocks.db')

bks = conn.execute('''select distinct bk from STOCKBASIC''').fetchall()

print(len(bks))
for bk in bks:
    print(bk)

conn.close()



In [ ]:
import sqlite3

def select_mv_count(mv):
    conn = sqlite3.connect('../stocks.db')
    ret = conn.execute('''select count(*) from STOCKBASIC where marketvalue > {}'''.format(mv)).fetchall()
    return ret[0][0]



mv = [10000,2000, 1000, 500, 100, 50, 10, 0]

# 统计各种市值的数量
mvcount = [select_mv_count(v) for v in mv]
with open('temp/day_marketvalue.csv', 'w') as fs:
    fs.write('marketvalue, count\n')

    for i in range(len(mv)):
        mvi = mv[i]
        mvcounti = mvcount[i]
        print('mv > {} count {}'.format(mvi, mvcounti))
        # if i > 0:
        #     print('{} > mv > {} count {}'.format(mv[i - 1], mvi, mvcounti - mvcount[i - 1]))
        fs.write('{},{}\n'.format(mvi, mvcounti))
        pass


In [ ]:
import matplotlib.pyplot as plt

#plt.bar(mv, mvcount)
plt.plot(range(len(mvcount)), mvcount)
plt.grid()
plt.show()



